# Feature Extraction HSV dan GLCM

In [1]:
import os
import numpy as np
import pandas as pd
from skimage.feature import graycomatrix, graycoprops
from skimage.color import rgb2gray, rgb2hsv
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from scipy.stats import skew, entropy


In [2]:
# Inisialisasi daftar untuk menyimpan fitur
feature_list = []
label_list = []

# Path folder yang berisi citra-citra dari 5 kelas 
folder_path = "../Dataset/"

In [3]:
# Daftar sudut arah yang ingin digunakan (0, 45, 90, 135 derajat)
angles = [0, 45, 90, 135]

for class_name in os.listdir(folder_path):
    class_folder = os.path.join(folder_path, class_name)
    if os.path.isdir(class_folder):
        for filename in os.listdir(class_folder):
            if filename.endswith(".jpg"): 
                image_path = os.path.join(class_folder, filename)
                # Baca citra, ubah ke grayscale, dan resize 
                image = Image.open(image_path).convert('RGB')
                image = np.array(image)
                gray_image = rgb2gray(image)
                gray_image = (gray_image * 255).astype(np.uint8)
                gray_image = Image.fromarray(gray_image)
                gray_image = gray_image.resize((300, 300), Image.ANTIALIAS)
                gray_image = np.array(gray_image)
                # Hitung GLCM untuk setiap sudut
                glcm_features = []
                for angle in angles:
                    glcm = graycomatrix(gray_image, [1], [angle], levels=256, symmetric=True, normed=True)
                    contrast = graycoprops(glcm, 'contrast')[0, 0]
                    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
                    correlation = graycoprops(glcm, 'correlation')[0, 0]
                    energy = graycoprops(glcm, 'energy')[0, 0]
                    glcm_features.extend([contrast, homogeneity, correlation, energy])
                # Konversi citra RGB ke HSV
                hsv_image = rgb2hsv(image)
                h, s, v = hsv_image[:, :, 0], hsv_image[:, :, 1], hsv_image[:, :, 2]
                # Ekstraksi fitur dari komponen HSV
                mean_h = np.mean(h)
                mean_s = np.mean(s)
                mean_v = np.mean(v)
                skew_h = skew(h.flatten())
                skew_s = skew(s.flatten())
                skew_v = skew(v.flatten())
                entropy_h = entropy(h.flatten())
                entropy_s = entropy(s.flatten())
                entropy_v = entropy(v.flatten())
                std_h = np.std(h)
                std_s = np.std(s)
                std_v = np.std(v)
                # Simpan fitur dan label
                feature_list.append(glcm_features + [mean_h, mean_s, mean_v, skew_h, skew_s, skew_v, entropy_h, entropy_s, entropy_v, std_h, std_s, std_v])
                label_list.append(class_name)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_20140\2752251329.py:16: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  gray_image = gray_image.resize((300, 300), Image.ANTIALIAS)


In [ ]:
X = np.array(feature_list)
y = np.array(label_list)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
# Buat DataFrame
column_names = [f'{feature}_{angle}' for feature in ['contrast', 'homogeneity', 'correlation', 'energy'] for angle in angles]
column_names += ['mean_h', 'mean_s', 'mean_v', 'skew_h', 'skew_s', 'skew_v', 'entropy_h', 'entropy_s', 'entropy_v', 'std_h', 'std_s', 'std_v']
df = pd.DataFrame(data=X, columns=column_names)
df['label'] = y_encoded

In [ ]:
# Simpan Dataframe ke dalam file CSV
file_name = "../notebook/result/output_combine_GLCM_HSV.csv"  
df.to_csv(file_name, index=False)